# S12 Iterables & Iterators

## Iterating Collections

- když iterujeme sekvence ..getitem(0), geitem(1) apod..
- stačí nám collection, container - prostě "bucket of items"
- a stačí nám funkce get next item -> základní koncept iterace
- může a nemusí záležet na pořadí výběru
- tzn. stačí nám next metoda pro iterátor - nemusí se nutně jednat o sekvenci

In [1]:
s = {"x","y","b","c","a"}

In [2]:
for item in s:
    print(item)

y
x
a
c
b


Tzn. máme iterátor a nezáleží na pořadí.

In [9]:
class Squares:
    def __init__(self):
        self.i = 0 #poslouží jako index
        
    def next_(self):
        result = self.i **2
        self.i += 1
        return result

In [10]:
sq = Squares()

In [11]:
sq.next_()

0

In [12]:
sq.next_()

1

Náš prvotní generátor. Prozatím neumí nic extra. Jak jej restartuji? 

In [17]:
sq = Squares()

In [18]:
sq.next_(), sq.next_()

(0, 1)

Například takto. Samozřejmě mohl bych i metodou apod.

In [20]:
sq = Squares()
for i in range(5):
    print(sq.next_())

0
1
4
9
16


Nyní hurá upravit naší classu.

In [21]:
class Squares:
    def __init__(self, length):
        self.i = 0 #poslouží jako index
        self.length = length
    
    def __len__(self):
        return self.length
    #TAK a nyní jakmile mám délku a mohu 
    #definovat délku mám stop
    
    def next_(self):
        if self.i >= self.length:
            raise StopIteration
            #v podstatě délka a i mi zajistí vyčerpání iterátoru
        else:
            result = self.i **2
            self.i += 1
            return result

In [22]:
sq = Squares(3)

In [23]:
len(sq)

3

In [24]:
sq.next_()

0

In [25]:
sq.next_()

1

In [26]:
sq.next_()

4

In [27]:
sq.next_()

StopIteration: 

Vidíme, že naše classa je již vyčerpaná. Viz. naše len metoda a podmínka pro next.

In [29]:
sq = Squares(3) #znovu jí restartujeme

In [31]:
sq = Squares(10)
while True:
    try:
        print(sq.next_())
    except StopIteration:
        break #jelikož vím jaký error má nasta při vyčerpání - ignoruji ji
#Tadáá a máme náš perfektní looping 

0
1
4
9
16
25
36
49
64
81


Naše sq je právě "exhausted". Nemusíme však psát next metodu - máme na to dunder metodu.

In [32]:
class Squares:
    def __init__(self, length):
        self.i = 0 #poslouží jako index
        self.length = length
    
    def __len__(self):
        return self.length
    
    def __next__(self): #změnil jsem pouze na dunder next
        if self.i >= self.length:
            raise StopIteration
        else:
            result = self.i **2
            self.i += 1
            return result

Nemusím tak psát _next() ale jen next

In [41]:
sq = Squares(3)

In [42]:
next(sq),next(sq),next(sq)

(0, 1, 4)

Stále však potřebujeme zajistit aby naše classa uměla for loop. Tzn. hurá zase směrem k sekvencím a tedy ke getiitem?

In [43]:
import random

In [44]:
class RandomNumbers:
    def __init__(self, length, *, range_min=0, range_max = 10):
        self.length = length
        self.range_min = range_min
        self.range_max = range_max
        self.num_requested = 0
        
    def __len__(self):
        return self.length
    
    def __next__(self):
        if self.num_requested >= self.length:
            raise StopIteration
        else:
            self.num_requested +=1
            return random.randint(self.range_min, self.range_max)

#máme prostě jen classu které má délku minimum a maximum
#a náhodně generuje něco z range
#ale máme omezený počet vyžádání si čísla
#něco jako losuj jen xkrát :)

In [48]:
numbers = RandomNumbers(2)

In [49]:
next(numbers), next(numbers)

(10, 8)

Náhodně jsme vygenerovalo 10 a 8 - na potřebí bych jíž dostal stop iteration error.

In [51]:
numbers = RandomNumbers(10)
while True:
    try:
        print(next(numbers))
    except StopIteration:
        break

8
5
6
7
3
3
9
4
4
3


O co zde jde, nemáme sekvenční typ ale neznamená to, že nemůžeme iterovat skrze něj. Jen přes next metodu nelze použít for loop. Musíme takto imrprovizovaně.

## Iterators

- v podstatě musí splnit iterator protocol aby byl plněhodnotým iteratorem
- první metoda je již výše zmíněná metoda _next_
- a další metoda je _iter_ která v podstatě vrátí sama sebe
- a tím pádem vytvoří jakoby znovu objekt
- a python díky tomu ví, že plně splňuje např. moje classa iterační protokol
- a tadáá...už plně funguje for loop, list comprehension apod.

In [53]:
class Squares:
    def __init__(self, length):
        self.length = length
        self.i = 0
        
    def __len__(self):
        return self.length
        
    def __next__(self):
        if self.i >= self.length:
            raise StopIteration
        else:
            result = self.i **2
            self.i +=1
            return result
        
    def __iter__(self):
        return self #nic více prostě navrátí sám sebe

In [59]:
sq = Squares(5)

In [55]:
for item in sq:
    print(item)

0
1
4
9
16


Již mohu používat for loop. V podstatě díky iter funkci, splnili jsme iterační protkol.

In [56]:
for item in sq:
    print(item)

I tak ale máme již vyčerpán, a musíme vytvořit novou jinak mám prázdno.

In [64]:
sq = Squares(5)

In [60]:
l = [(item,item+1) for item in sq]

In [61]:
l

[(0, 1), (1, 2), (4, 5), (9, 10), (16, 17)]

In [62]:
l = [(item,item+1) for item in sq]

In [63]:
l

[]

Znovu již máme prázdný list, konec iterace a vše se mi vyčerpalo. Nyní tedy musíme ještě vyladit classu tak aby se uměla sama o sobě "vyresetovat".

In [73]:
class Squares:
    def __init__(self, length):
        self.length = length
        self.i = 0
        
    def __len__(self):
        return self.length
        
    def __next__(self):
        print("__next__ called")
        if self.i >= self.length:
            raise StopIteration
        else:
            result = self.i **2
            self.i +=1
            return result
        
    def __iter__(self):
        print("__iter__called")
        return self #nic více prostě navrátí sám sebe

In [74]:
sq = Squares(5)

In [70]:
for item in sq:
    print(item)

__iter__called
__next__ called
0
__next__ called
1
__next__ called
4
__next__ called
9
__next__ called
16
__next__ called


Takže při iteraci naskočí iter protokol, a poté next called. While loop volá pouze next bez iter metody.

## Iterator & Iterables

- nyní to již finálně propojíme

In [75]:
class Cities:
    def __init__(self):
        self._cities = ["Paris","Berlin","Rome","Madrid","London"]
        self._index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._index >= len(self._cities):
            raise StopIteration
        else:
            item = self._cities[self._index]
            self._index +=1
            return item

In [76]:
cities = Cities()

In [77]:
type(cities)

__main__.Cities

A disponuje iteračním protokolem.

In [78]:
list(enumerate(cities))

[(0, 'Paris'), (1, 'Berlin'), (2, 'Rome'), (3, 'Madrid'), (4, 'London')]

In [79]:
list(enumerate(cities))

[]

Znovu, nefunkční dokud bychom jej nerestartovali.

In [81]:
class Cities:
    def __init__(self):
        self._cities = ["Paris","Berlin","Rome","Madrid","London"]
        self._index = 0
    
    def __len__(self):
        return len(self._cities)
#splitnu to - potřebuji mít pouze classu Cities, která drží to co potřebujeme.

In [82]:
class CityIterator:
    def __init__(self, city_obj):
        self._city_obj = city_obj
        self._index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._index >= len(self._city_obj): #jelikož cities class má len
            #mohu takto zapsat jinak bych musel ._city_obj.cities
            raise StopIteration
        else:
            item = self._city_obj._cities[self._index]
            #není to list ale classa - obdrží v parametru
            #classu cities ..tak zní musí vydolovat cities :)
            #jinak by stačil zápis s indexem ..
            self._index +=1
            return item

In [83]:
cities = Cities()

In [86]:
city_iterator = CityIterator(cities)

In [87]:
for city in city_iterator:
    print(city)

Paris
Berlin
Rome
Madrid
London


Ale už nemusím dolovat data o městech - ty se načítají. Ale stále se nám iterátor vyčerpá.

In [91]:
city_iterator = CityIterator(cities) #tuhle část stále musíme dělat..

Musíme implementovat ITERABLE PROTOKOL - NIKOLIV ITERATOR :)

In [93]:
class Cities:
    def __init__(self):
        self._cities = ["Paris","Berlin","Rome","Madrid","London"]
        self._index = 0
    
    def __len__(self):
        return len(self._cities)
    
    def __iter__(self):
        return CityIterator(self)
    
    #cities navrací sama sebe ale v podobě iteratoru
    #tzn..classa co sama o sobě vrací sama sebe skrze iterator

A nyní jsme to perfektně přetočily :).

In [94]:
cities = Cities()

In [95]:
for city in cities:
    print(city)

Paris
Berlin
Rome
Madrid
London


In [96]:
for city in cities:
    print(city)

Paris
Berlin
Rome
Madrid
London


Nyní to zkusíme trochu zdokumentovat, printěním.

In [97]:
class Cities:
    def __init__(self):
        self._cities = ["Paris","Berlin","Rome","Madrid","London"]
        self._index = 0
    
    def __len__(self):
        return len(self._cities)
    
    def __iter__(self):
        print("Cities __iter__called")
        return CityIterator(self)

In [100]:
class CityIterator:
    def __init__(self, city_obj):
        print("CityIterator new object")
        self._city_obj = city_obj
        self._index = 0
        
    def __iter__(self):
        print("CityIterator __iter__ called")
        return self
    
    def __next__(self):
        print("Cityiterator next called")
        if self._index >= len(self._city_obj):
            raise StopIteration
        else:
            item = self._city_obj._cities[self._index]

            self._index +=1
            return item

In [101]:
cities = Cities()

In [102]:
for city in cities:
    print(city)

Cities __iter__called
CityIterator new object
Cityiterator next called
Paris
Cityiterator next called
Berlin
Cityiterator next called
Rome
Cityiterator next called
Madrid
Cityiterator next called
London
Cityiterator next called


In [103]:
#Cities __iter__called
#CityIterator new object
#Cityiterator next called

Takže tyhle první řádky jsou skvělé. Tvorba nového objektu je tvorba nového iteratoru. Ten si zde dotváří sám.

In [104]:
city_iter_1 = cities.__iter__()

Cities __iter__called
CityIterator new object


Přidáme sekvenční protokol k naší classa a s nestíme jí. Co bude používat spíše ITER nebo GETITEM?

In [105]:
class Cities:
    def __init__(self):
        self._cities = ["Paris","Berlin","Rome","Madrid","London"]
        self._index = 0
    
    def __len__(self):
        return len(self._cities)
    
    def __iter__(self):
        print("Cities __iter__called")
        return self.CityIterator(self)
    
    def __getitem__(self,s):
        print("getting item")
        return self._cities[s]
    
    class CityIterator:
        def __init__(self, city_obj):
            print("CityIterator new object")
            self._city_obj = city_obj
            self._index = 0

        def __iter__(self):
            print("CityIterator __iter__ called")
            return self

        def __next__(self):
            print("Cityiterator next called")
            if self._index >= len(self._city_obj):
                raise StopIteration
            else:
                item = self._city_obj._cities[self._index]

                self._index +=1
                return item

In [107]:
cities = Cities()

In [109]:
cities[0]

getting item


'Paris'

In [110]:
for city in cities:
    print(city)

Cities __iter__called
CityIterator new object
Cityiterator next called
Paris
Cityiterator next called
Berlin
Cityiterator next called
Rome
Cityiterator next called
Madrid
Cityiterator next called
London
Cityiterator next called


Preferuje iterační protokol před getitem. Pokud by nenašel iterační protokol tak poté by mohl iterovat pouze přes getitem protkol. Takto funguje většina datových typů.

In [111]:
l = [1,2,3,4]

In [112]:
iter(l)

In [113]:
l.__iter__

<method-wrapper '__iter__' of list object at 0x000002111625ECC0>

In [114]:
l.__getitem__

<function list.__getitem__>

In [116]:
l_iter = iter(l)

In [117]:
for i in l_iter:
    print(i)

1
2
3
4


In [119]:
next(l_iter)

StopIteration: 

Takže jsem si vytvořil list, vydoloval z něj jeho iterátor, vyčerpal ho a nechal vyhodit chybu - tu co jsme psali i my. Tzn. vše je to založeno opravdu na stejných typech.

## Consuming Iterators Manually

In [120]:
s = "I sleep all night, and i work all day"

In [123]:
iter_s = iter(s) #vydoluji si jen iterátor - mohu jej vyčerpat

In [124]:
iter_s

In [125]:
next(iter_s)

'I'

In [126]:
next(iter_s)

' '

In [127]:
next(iter_s)

's'

In [148]:
from collections import namedtuple
cars = []

with open("cars.csv") as file:
    row_index = 0 #eviduji si řádky - nechci načítat celý ale po řádku
    for line in file:
        if row_index == 0:
            #header row - odmažu konec a odmažu středník
            headers = line.strip("\n").split(";")
            Car = namedtuple("Car", headers)
            #vytvárím named tuple z hlavičky
        elif row_index ==1:
            #data typ řádek
            data_types = line.strip("\n").split(";")
        else:
            #data
            data = data_types = line.strip("\n").split(";")
            data = cast_row(data_types, data)
            #zde volám naší super funcki
            car = Car(*data) #vytvářím named tuply
            cars.append(car) #a všechny je dávám do listu
        row_index += 1
        

In [149]:
cars[0:4]

[Car(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US'),
 Car(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US'),
 Car(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US'),
 Car(Car='AMC Rebel SST', MPG='16.0', Cylinders='8', Displacement='304.0', Horsepower='150.0', Weight='3433.', Acceleration='12.0', Model='70', Origin='US')]

In [144]:
def cast(data_type, value):
    """FUNKCE PRO ZMĚNU FORMÁTU"""
    if data_type == "DOUBLE":
        return float(value)
    elif data_type =="INT":
        return int(value)
    else:
        return str(value)

In [145]:
def cast_row(data_types, data_row):
    return [cast(data_type,value)
    for data_type, value in zip(data_types, data_row)]
#volám funkci - která potřebuje data typ a řádek
#kde řádek budu vždycky jiný a data typ je shodný
#a ta funkce vytváří listy, které se tvoří pomocí funkce cast

Teď to zkusíme trochu jinak.

In [150]:
from collections import namedtuple

In [151]:
cars = []

In [158]:
with open("cars.csv") as file:
    file_iter = iter(file) #vydoluji si ze souboru iterátor a půjde vyčerpat
    headers = next(file_iter) #doluji první
    data_types = next(file_iter) #a druhý
    for line in file_iter:
        pass
        #normální loop ale začnu až od 3 řádku
        #A UŽ BYCH VŠE DODĚLAT..
        #MOHL BYCH DOLOVAT POSTUPNĚ ..nebo náročněji apod..

## Cyclic Iterators

Tento příklad vzít pomaleji a jistěji.

In [159]:
# 1 2 3 4 5 6 7 8 9
# N S W E
# 1N 2S 3W 4E 5N 6S ..

In [165]:
class CyclicIterator:
    def __init__(self,lst):
        self.lst = lst
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        result = self.lst[self.i % len(self.lst)]
        #díky mod délce listu se točíme vždy v listu
        #dokola - 0 % 4 = 0, 1%4 = 1..4%4 = 0 atd.. :)
        self.i += 1
        return result

In [166]:
iter_cycl = CyclicIterator("NSWE")

In [167]:
for _ in range(10):
    print(next(iter_cycl))

N
S
W
E
N
S
W
E
N
S


- Můžeme tedy vidět, že trik s modem +- vše pořešil a můžeme cyclicky loopovat.

In [170]:
numbers = range(1,11)

In [171]:
list(numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [172]:
iter_cycl = CyclicIterator("NSWE")

In [175]:
list(zip(list(numbers), iter_cycl))
#nejdříve to zkusíme pomocí zip funkce

[(1, 'W'),
 (2, 'E'),
 (3, 'N'),
 (4, 'S'),
 (5, 'W'),
 (6, 'E'),
 (7, 'N'),
 (8, 'S'),
 (9, 'W'),
 (10, 'E')]

In [176]:
n = 10
iter_cycl = CyclicIterator("NSWE") #vytvořím si můj cyclickej iterátor
for i in range(1, n+1): #a projedu ho v rozmezí n
    direction = next(iter_cycl) #jedna část ne N...E
    print(f"{i}{direction}") #druhá řást je i - v podstatě index
#nyní trochu hardcod

1N
2S
3W
4E
5N
6S
7W
8E
9N
10S


In [179]:
#nyní pomocí LH
n = 10
iter_cycl = CyclicIterator("NSWE")
items = [str(i) + next(iter_cycl) for i in range(1,n+1)]
#prostě jen sčítám string ička - 1...n a hodnotu z iterátoru

In [180]:
items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

A nyní to samé znovu pomocí zipu a LH.

In [182]:
n = 10
iter_cycl = CyclicIterator("NSWE")
[str(number) + direction for number, direction
 in zip(range(1, n+1), iter_cycl)]
#v podstatě používám hodnoty které získávám zipováním

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

A nyní znovu zip ale trochu jinak.

In [184]:
list(zip(range(1,11), "NSWE" *30))

[(1, 'N'),
 (2, 'S'),
 (3, 'W'),
 (4, 'E'),
 (5, 'N'),
 (6, 'S'),
 (7, 'W'),
 (8, 'E'),
 (9, 'N'),
 (10, 'S')]

V podstatě zvládá všechny ostatní věci samo o sobě. A nyní další cyclický iterátor pomocí zipu a LH.

In [185]:
items = [str(number) + direction for number, direction 
 in zip(range(1, n+1), "NSWE" * (n // 4  + 1 ))]

In [186]:
items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

A nyní cylic iterátor built in.

In [187]:
import itertools

In [188]:
n = 10

In [189]:
iter_cycl = CyclicIterator("NSWE")

In [191]:
[f"{i}{next(iter_cycl)}" for i in range(1, n+1)]
#LH jsou opravdu bláznivá, všemocný to nástroj

['1W', '2E', '3N', '4S', '5W', '6E', '7N', '8S', '9W', '10E']

In [192]:
n = 10
iter_cycl = itertools.cycle("NSWE")
[f"{i}{next(iter_cycl)}" for i in range(1, n+1)]

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

In [193]:
help(itertools.cycle)

Help on class cycle in module itertools:

class cycle(builtins.object)
 |  cycle(iterable, /)
 |  
 |  Return elements from the iterable until it is exhausted. Then repeat the sequence indefinitely.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __setstate__(...)
 |      Set state information for unpickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



Naše fičura která by to měla zvládat bez nutnosti vytvářet si cyclic sekvenci.

In [194]:
s = {100, "a", "x", "X", 200} #můžeme to tedy zkusit na setu

In [211]:
#takže to zkusíme trochu rozebrat
class CyclicIterator:
    def __init__(self,iterable): #iterable je to co chceme iterovat např. "abc"
        self.iterable = iterable  #jen ho přiřazujeme samy sobě
        self.iterator = iter(self.iterable) #a zakládáme iterator právě z naší iterable
        #iterator inicializujeme jen jednou
        
    def __iter__(self):
        return self #klasika jen abychom zajistili protkol
    
    def __next__(self): #a nyní celá show
        try: #díky try se vyhneme erroru ..
            item = next(self.iterator) #voláme iterator při next
            #pokud nachází item je vše ok a jede dále
        except StopIteration: #pokud ale najede na konec
            self.iterator = iter(self.iterable) #znovu jakoby nahodí motor :)
            #tzn. vygeneruje si iterátor ještě jednou
            item = next(self.iterator) #ale ihned zavolá další item
        finally:
            return item #item se vždy vrací jak po konci try
            #tak po konci except - vždy nějaký musí být

In [212]:
iter_cycl = CyclicIterator("abc")

In [214]:
for i in range(10):
    print(i, next(iter_cycl))
#a tudíž máme cyklický iterátor pro cokoliv a stačí mu zadat jen range

0 b
1 c
2 a
3 b
4 c
5 a
6 b
7 c
8 a
9 b


## Lazy Iterables

Často se používá u class.

In [215]:
import math

In [233]:
class Circle:
    def __init__(self, r):
        print("init circle")
        self.radius = r
    
    @property
    def radius(self):
        print("radius property")
        return self._radius
    
    @radius.setter
    def radius(self, r):
        print("radius and area")
        self._radius = r
        self.area = math.pi * (r **2)

In [234]:
c = Circle(1)

init circle
radius and area


In [236]:
c.radius
c.area

radius property


3.141592653589793

In [237]:
c.radius = 2

radius and area


In [238]:
c.area

12.566370614359172

Zajímavé je že při inciializaci se kalkuluje radius i area

Pořádně nevím co dělá property a radius, je spíše na OOP lekci. Logika je tady, že mám lazy variable. Tzn. area není v initu ale i tak je kalkulován a je vždycky překalkulovaná hodnota i pokud jen změním radius - viz. výše. Budeme chtít zajisti aby se area počítala jen pokud je to vyžadováno.

In [239]:
class Circle:
    def __init__(self, r):
        print("init")
        self.radius = r
    
    @property
    def radius(self):
        print("radius property")
        return self._radius
    
    @radius.setter
    def radius(self, r):
        print("radius setter")
        self._radius = r
        
    @property
    def area(self):
        print("Calculating are..")
        return math.pi * (self.radius ** 2)

In [241]:
c = Circle(1) #setter naskakuje při initu tedy

init
radius setter


In [232]:
c.area #kalkulace probíhá jen pokud chci area..

Calculating are..


3.141592653589793

Nyní se nám kalkuluje area až pokud si jí žádáme. ALE bylo by lepší aby se kalkulovala jen pokud se mění radius :).

In [245]:
class Circle:
    def __init__(self, r):
        print("init")
        self.radius = r
        self._area = None #počátek 
    
    @property
    def radius(self):
        print("radius property")        
        return self._radius
        
    
    @radius.setter
    def radius(self, r):
        print("radius setter and also i am deleting area")
        self._radius = r
        self._area = None
        #pokud změním r - musím znovu počítat area 
        #proto jej nuluji!
        
    @property
    def area(self):
        #a zde je to jednoduché pokud are je none - tak jej počítám
        #pokud není none - tak má hodnotu a jen já vrátím
        if self._area is None:
            print("Calculating are..")
            self._area = math.pi * (self.radius ** 2)
        else:
            print("I didn't have to calculate area")
        return self._area

In [252]:
c = Circle(1)

init
radius setter and also i am deleting area


In [253]:
c.area #prvně request na area s kalkulaci

Calculating are..
radius property


3.141592653589793

In [254]:
c.area #nyní bez kalkulace

I didn't have to calculate area


3.141592653589793

In [256]:
c.radius = 2 #měním radius a nuluji area

radius setter and also i am deleting area


In [257]:
c.area #a musím znovu počítat

Calculating are..
radius property


12.566370614359172

In [258]:
c.area #a nyní už zase ne :)

I didn't have to calculate area


12.566370614359172

No dokonalost. Nyní stejná logika pro sekvence.

In [271]:
class Factorials:        
    def __iter__(self):
        return self.FactIter()
    #nestěná classa - kde první classa navrací druhou
    #ale jen pokud si vyžádáme její iterátor 
    #jinak by musel být init s délkou
    class FactIter:
        def __init__(self):
            self.i = 0
            #ten si jen hlídá i - aby mohl volat další itemy
            
        def __iter__(self):
            return self #klasika
        
        def __next__(self):
            result = math.factorial(self.i) #pouze funkce 
            self.i += 1
            return result
        #ona tahle funkce umí volat jen podle next ..nic jí nejde přiřadit

In [265]:
facts = Factorials()

In [266]:
fact_iter = iter(facts)

In [267]:
next(fact_iter)

1

In [268]:
next(fact_iter)

1

In [269]:
next(fact_iter)

2

In [270]:
next(fact_iter)

6

Nemá délku, nemá nic k definování. Jediný přínos je, že v něm je v podstatě iterátor sám o sobě - který vydolujeme. A je to lazy iterátor protože se dotváří při vyžádání.

## Python a Built-In Iterables a Iterátory

In [277]:
r = range(5)

In [273]:
"__iter__" in dir(r)

True

In [274]:
"__next__" in dir(r)

False

In [275]:
next(r)

TypeError: 'range' object is not an iterator

Není to iterátor ale je iterable.

In [278]:
for s in r:
    print(s)

0
1
2
3
4


In [279]:
z = zip([1,2,3], "abc")

In [282]:
"__iter__" in dir(z), "__next__" in dir(z)

(True, True)

In [283]:
list(z)

[(1, 'a'), (2, 'b'), (3, 'c')]

In [284]:
list(z)

[]

Jako každý iterátor, když jsme jej projeli tak se vyčerpal :). A soubory jsou též iterátory.

In [287]:
f = open("cars.csv")
print(next(f))
print(f.__next__())
f.close()

Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin

STRING;DOUBLE;INT;DOUBLE;DOUBLE;DOUBLE;DOUBLE;INT;CAT



In [291]:
#soubor je iterátor - vyčerpá se..
with open("Cars.csv") as f:
    print(type(f))
    print("__iter__" in dir(f)) 
    print("__next__" in dir(f))
    print(iter(f) is f) #pokud je iterátor je roven sám sobě!

<class '_io.TextIOWrapper'>
True
True
True


- TZN ITERÁTOR JE ROVEN SÁM SOBĚ! iter(f) is f

In [292]:
l = [1,2,3]

iter(l) is l

False

- Ale l iter není l - PROČ? Je to ITERABLE - nikoliv iterátor
- Neboli generuje nový objekt vždycky - nový iterátor

In [293]:
origins = set()

In [294]:
with open("cars.csv") as f:
    rows = f.readlines()
for row in rows[2:]:
    origin = row.strip("\n").split(";")[-1]
    #doluji poslední sloupec
    origins.add(origin)
    
print(origins) #dávám to do setu proto málo výsledků

{'Europe', 'Japan', 'US'}


Jenže musím nahrávat celý soubor naráz.

In [296]:
origins = set()

with open("cars.csv") as f:
    next(f)
    next(f)#přeskočím dva řádky
    for row in f:
        origin = row.strip("\n").split(";")[-1]
        origins.add(origin)
print(origins)   
#takto nemusím číst soubor

{'Europe', 'Japan', 'US'}


Enumerator je též iterátor.

In [297]:
e = enumerate("Python Rocks!")

In [298]:
iter(e) is e

True

In [299]:
list(e)

[(0, 'P'),
 (1, 'y'),
 (2, 't'),
 (3, 'h'),
 (4, 'o'),
 (5, 'n'),
 (6, ' '),
 (7, 'R'),
 (8, 'o'),
 (9, 'c'),
 (10, 'k'),
 (11, 's'),
 (12, '!')]

In [303]:
list(e)

[]

## Sorting Iterables

In [304]:
import random

In [312]:
random.seed(0) #seed mi prostě zajisí stejná čísla - a zajímavé je že i ostatním :)
#tzn můj seed0 dodá stejnej výsledek jako v přednáškách ...
for _ in range(10):
    print(random.randint(1,10))

7
7
1
5
9
8
7
5
8
6


In [345]:
class RandomInts:
    def __init__(self, length, *, seed = 0, lower=0, upper=10):
        self.length = length
        self.seed = seed
        self.lower = lower
        self.upper = upper
        #prostě jen init základních věcí
        
    def __len__(self): #délka nic extra
        return self.length
    
    def __iter__(self): #náš iter funkce přeposílá 
        #info do randomiteratoru
        return self.RandomIterator(self.length,
                                   seed=self.seed,
                                   lower = self.lower,
                                   upper = self.upper)
        
        
    class RandomIterator:
        def __init__(self, length, *, seed, lower, upper):
            self.length = length
            self.lower = lower
            self.upper = upper
            self.num_requests = 0
            random.seed(seed)
            #má svojí inicializaci - navíc počet rquestů
        
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.num_requests >= self.length:
                raise StopIteration
            else: #a iterace probíhá zde podle hranic 
                #a vytváří náhodná čísla
                result = random.randint(self.lower, self.upper)
                self.num_requests += 1
                return result

In [347]:
randoms = RandomInts(10)

In [348]:
for num in randoms:
    print(num)

6
6
0
4
8
7
6
4
7
5


Máme pořád stejná čísla, protože máme stejnej seed. Musím si nadefinovat nový seed, poté bude vše Ok.

In [349]:
randoms = RandomInts(10, seed=None)

In [352]:
for num in randoms:
    print(num) #takhle to poté frčí jak chci

5
1
5
4
7
10
6
9
1
1


In [355]:
randoms = RandomInts(10)

In [356]:
list(randoms)

[6, 6, 0, 4, 8, 7, 6, 4, 7, 5]

In [357]:
sorted(randoms)

[0, 4, 4, 5, 6, 6, 6, 7, 7, 8]

Sorted obecně funguje pro iterables a ne jenom pro sekvence.

## Iter funkce

In [358]:
l = [1,2,3,4]

In [359]:
l_iter = iter(l)

In [360]:
next(l_iter)

1

V podstatě máme iter funkci a nemusíme se poté párat se zaváděním protokolu?

In [365]:
class Squares:
    def __init__(self, n):
        self._n = n
        
    def __len__(self):
        return self._n
    
    def __getitem__(self,i):
        if i>= self._n:
            raise IndexError
        else:
            return i**2
#sekvenční typ easy mode

In [366]:
sq = Squares(5)

In [367]:
for i in sq:
    print(i)

0
1
4
9
16


In [368]:
sq_iter = iter(sq)

In [369]:
type(sq_iter)

iterator

In [370]:
"__next__" in dir(sq_iter)

True

In [371]:
next(sq_iter)

0

GetItem metod jak se zdá plnohodnotně zastoupí celý iterable protokol. Zkusíme to trochu probádat.

In [391]:
class Squares:
    def __init__(self, n):
        self._n = n
        
    def __len__(self):
        return self._n
    
    def __getitem__(self,i):
        if i>= self._n:
            raise IndexError
        else:
            return i**2

In [392]:
class SquaresIterator:
    def __init__(self, squares):
        self._squares = squares
        self._i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        """implementujeme iter prot"""
        if self._i >= len(self._squares):
            raise StopIteration
        else:
            result = self._squares[self._i]
            self._i += i
            return result

In [387]:
sq = Squares(5)

In [388]:
sq_iterator = SquaresIterator(sq)
#mému square iterátoru vkládám square classu - která ale sama o sobě
#disponuje geiitem a už sme skrze ní mohli iterovat

In [393]:
print(next(sq_iterator))

StopIteration: 

Classa výše může být v podstatě obecně použitelná na všechno. A v podstatě nám to ukazuje to co dělá iter funkce.

In [394]:
class SimpleIter:
    def __init__(self):
        pass
    
    def __iter__(self):
        return "Nope" #zkoušíme co se stane

In [395]:
s = SimpleIter()

In [398]:
"__iter__" in dir(s)
#METODU TO SAMOZŘEJMĚ NAJDE :D
#ALE NEVÍ ŽE JE ŠPATNÁ ..

True

In [399]:
iter(s)

TypeError: iter() returned non-iterator of type 'str'

Je tedy lepší volat funkci iter pro ověření jestli mám iterator.

In [402]:
def is_iterable(obj):
    try:
        iter(obj)
        return True
    except TypeError:
        return False

In [403]:
is_iterable(s)

False

In [404]:
is_iterable(Squares(5))

True

In [405]:
is_iterable("abc")

True

O co jde, můžeme tak mít moc pěknou kontrolu - error handling.

In [407]:
obj = 100
if is_iterable(obj):
    for i in obj:
        print(i)
    else:
        print("Error: obj is not iterable")

Je snadnější žádat o odpuštění, než žádat o svolení. Error handling moundro.

## Iterování Callables

In [408]:
def counter():
    i = 0
    
    def inc():
        nonlocal i
        i += 1
        return i
    return inc
#counter nic víc

In [409]:
cnt = counter()

In [410]:
cnt()

1

In [411]:
cnt()

2

A nyní iterátor pro counter.

In [412]:
class CounterIterator:
    def __init__(self, counter_callable):
        self.counter_callable = counter_callable
        
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.counter_callable()
    
#nekonečný iterátor - pozor

In [415]:
cnt = counter() #funkce counter

In [416]:
cnt_iter = CounterIterator(cnt) #classa co obdrží jako parametr funkci

In [417]:
for _ in range(5):
    print(next(cnt_iter))

1
2
3
4
5


Musím volat next - jinak by nastalaa infinite loop. Mírně to upravíme.

In [429]:
class CounterIterator:
    def __init__(self, counter_callable, sentinel):
        self.counter_callable = counter_callable
        self.sentinel = sentinel
        
    def __iter__(self):
        return self
    
    def __next__(self):
        result = self.counter_callable() #spouští funkci 
        if result == self.sentinel:
            raise StopIteration
        else: #nastavujeme si hranici iterátoru
            return result
    
#nekonečný iterátor - pozor

In [430]:
cnt = counter()
cnt_iter = CounterIterator(cnt, 5) #se stopem 5

In [431]:
for c in cnt_iter:
    print(c)

1
2
3
4


In [433]:
next(cnt_iter) #není roven výsledek sentinelu proto
#je schopen může fungovat dále - musíme upravit 

7

In [434]:
class CounterIterator:
    def __init__(self, counter_callable, sentinel):
        self.counter_callable = counter_callable
        self.sentinel = sentinel
        self.is_consumed = False
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.is_consumed:
            raise StopIteration
        #dodáváme flag pro zjištění vyčerpání
        #iterátoru a následuje strop iterátoru
        else:
            result = self.counter_callable() #spouští funkci 
            if result == self.sentinel:
                self.is_consumed = True
                raise StopIteration
            else: #nastavujeme si hranici iterátoru
                return result

In [435]:
cnt = counter()
cnt_iter = CounterIterator(cnt, 5)

In [436]:
for c in cnt_iter:
    print(c)

1
2
3
4


In [437]:
next(cnt_iter)

StopIteration: 

Naše úprava funguje, díky self.is_consumed máme pozastavenou operaci. V podstatě máme podmínku jak pro strop tak pro vyčerpání. Nyní můžeme přepsat na více generic classu.

In [438]:
class CallableIterator:
    def __init__(self, callable_, sentinel):
        self.callable_ = callable_
        self.sentinel = sentinel
        self.is_consumed = False
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.is_consumed:
            raise StopIteration
        #dodáváme flag pro zjištění vyčerpání
        #iterátoru a následuje strop iterátoru
        else:
            result = self.callable_() #spouští funkci 
            if result == self.sentinel:
                self.is_consumed = True
                raise StopIteration
            else: #nastavujeme si hranici iterátoru
                return result

In [440]:
cnt = counter()
cnt_iter = CallableIterator(cnt, 5)

In [441]:
for c in cnt_iter:
    print(c)

1
2
3
4


O co jde? Nyní mám iterátor pro funkce.

In [442]:
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator
    
    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



iter(callable, sentinel) -> iterator --> to co momentálně děláme

In [445]:
cnt = counter()
cnt_iter = iter(cnt, 3) #bult in funkce

In [446]:
for c in cnt_iter:
    print(c)

1
2


In [447]:
next(cnt_iter)

StopIteration: 

Jde vidět že built in funkce iter funguje stejně. Nyní ukázka, že vše funguje obecně pro funkce.

In [448]:
import random

In [449]:
random.seed(0)

In [450]:
for i in range(10):
    print(i, random.randint(0,10))

0 6
1 6
2 0
3 4
4 8
5 7
6 6
7 4
8 7
9 5


In [451]:
random_iter = iter(lambda : random.randint(0,10), 8)

In [452]:
random.seed(0)

In [453]:
for num in random_iter:
    print(num)

6
6
0
4


In [454]:
def countdown(start=10):
    def run():
        nonlocal start
        start -= 1
        return start
    return run

In [458]:
takeoff = countdown(10)

In [459]:
for _ in range(5):
    print(takeoff())

9
8
7
6
5


In [460]:
for _ in range(5):
    print(takeoff())

4
3
2
1
0


Stále si pamatuje hodnoty, vynuluje se pouze při inicializaci takeoff.

## Delegating Iterators

In [461]:
from collections import namedtuple

In [462]:
Person = namedtuple("Person", "first last")

In [463]:
class Personnames:
    def __init__(self, persons):
        try:
            self._persons = [person.first.capitalize()
                             + " " + person.last.capitalize()
                             for person in persons]
        except (TypeError, AttributeError):
            self._persons = []

In [464]:
persons = [Person("michaeL", "Palin"), Person("eric", "idle"),
          Person("john", "cleese")]

In [466]:
person_names = Personnames(persons)

In [467]:
person_names._persons

['Michael Palin', 'Eric Idle', 'John Cleese']

Naše pěkná classa, opravila jména. V podstatě classa obdrží namedtuply person a upraví je. Nicméně nyní je chci mít jako iterable.

In [484]:
class Personnames:
    def __init__(self, persons):
        try:
            self._persons = [person.first.capitalize()
                             + " " + person.last.capitalize()
                             for person in persons]
        except (TypeError, AttributeError):
            self._persons = []
        #moc pěkné podchycení chyb a zajímavá inicializace
        #classy pomocí try - except..
            
    def __iter__(self):
        return iter(self._persons)
        #nemusím nic extra řešit - mám list takže deleguji
        #zodpovědnost za iteraci listu samotnému :)
        #v podstatě jen říkám - budeš iterovat ..jen podle
        #dat které máš ale jelikož to jsou listy je vše ok

In [485]:
person_names = Personnames(persons)

In [486]:
for person in person_names:
    print(person)

Michael Palin
Eric Idle
John Cleese


Obecně jakmile mám typ který je sám o sobě iterovatelný, delegace je nejvhodnější volba.

## Reverzní Iterace

Vytvoříme si zase krabičku karet.

In [488]:
_SUITS = ("Spades", "Heaarts", "Diamonds", "Clubs")

In [491]:
_RANKS = tuple(range(2,11)) + tuple("JQKA")

In [494]:
from collections import namedtuple

In [495]:
Card = namedtuple("Card", "rank suit")

In [524]:
class CardDeck:
    def __init__(self):
        self.length = len(_SUITS) * len(_RANKS)
        #délka čísel i značek u karet
        
    def __len__(self):
        return self.length
       
    def __iter__(self):
        return self.CardDeckIterator(self.length)
        #náš iterátor bude vytvářen iterátorem
    
    class CardDeckIterator:
        def __init__(self, length):
            self.length = length #dědí od classy výše
            self.i = 0
            
        def __iter__(self):
            return self #klasika
        
        def __next__(self):
            if self.i >= self.length: #délka je fixní
                #krabička karet je jasná
                raise StopIteration
            else:
                suit = _SUITS[self.i // len(_RANKS)]
                #díky tomu přiřazuje barvy od 1-4
                rank = _RANKS[self.i % len(_RANKS)]
                #a tohle od 1 do 14
                self.i += 1
                return Card(rank, suit)
                   

In [512]:
deck = CardDeck()

In [516]:
for card in deck:
    print(card)

Card(rank=2, suit='Spades')
Card(rank=3, suit='Spades')
Card(rank=4, suit='Spades')
Card(rank=5, suit='Spades')
Card(rank=6, suit='Spades')
Card(rank=7, suit='Spades')
Card(rank=8, suit='Spades')
Card(rank=9, suit='Spades')
Card(rank=10, suit='Spades')
Card(rank='J', suit='Spades')
Card(rank='Q', suit='Spades')
Card(rank='K', suit='Spades')
Card(rank='A', suit='Spades')
Card(rank=2, suit='Heaarts')
Card(rank=3, suit='Heaarts')
Card(rank=4, suit='Heaarts')
Card(rank=5, suit='Heaarts')
Card(rank=6, suit='Heaarts')
Card(rank=7, suit='Heaarts')
Card(rank=8, suit='Heaarts')
Card(rank=9, suit='Heaarts')
Card(rank=10, suit='Heaarts')
Card(rank='J', suit='Heaarts')
Card(rank='Q', suit='Heaarts')
Card(rank='K', suit='Heaarts')
Card(rank='A', suit='Heaarts')
Card(rank=2, suit='Diamonds')
Card(rank=3, suit='Diamonds')
Card(rank=4, suit='Diamonds')
Card(rank=5, suit='Diamonds')
Card(rank=6, suit='Diamonds')
Card(rank=7, suit='Diamonds')
Card(rank=8, suit='Diamonds')
Card(rank=9, suit='Diamonds')
C

Jak zajistím zpětnou iteraci? GetItem by to trochu vylepšil?

In [519]:
deck = list(CardDeck()) # převedu si list

In [520]:
deck[:-8:-1]

[Card(rank='A', suit='Clubs'),
 Card(rank='K', suit='Clubs'),
 Card(rank='Q', suit='Clubs'),
 Card(rank='J', suit='Clubs'),
 Card(rank=10, suit='Clubs'),
 Card(rank=9, suit='Clubs'),
 Card(rank=8, suit='Clubs')]

Ale musím si vytvářet list, jak to udělat lépe?

In [522]:
reversed_deck = reversed(CardDeck())

TypeError: 'CardDeck' object is not reversible

Classa potřebuje speciální metodu, aby byla schopná převést na reverzní řadu pomocí funkce reversed.

- musíme vytvořit v classe 1 reversed dunder
- a dále musíme mít iterátor který iteruje zpětně
- buď nový iteráor nebo modifikovaný

In [533]:
class CardDeck:
    def __init__(self):
        self.length = len(_SUITS) * len(_RANKS)
        
    def __len__(self):
        return self.length    
       
    def __iter__(self):
        return self.CardDeckIterator(self.length)
    
    def __reversed__(self):
        return self.CardDeckIterator(self.length, reverse = True)
        #moje reverse dunder a reverse flag
        #tzn. pokud použiji funkci reverse - pehodím vlajku na True
        
    class CardDeckIterator:
        def __init__(self, length, reverse = False):
            #vlajka je defaultně False..-přehodí jen reverse
            self.length = length
            self.reverse = reverse #přidáno nově
            self.i = 0
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.i >= self.length:
                raise StopIteration
            else:
                if self.reverse: #pokud chci zpětnou iteraci
                    index = self.length -1 - self.i  
                    #zpětná iterace pro větší představu lepší
                    #nakreslit ale teď nebudu řešit
                else:
                    index = self.i
                #místo self i používám index - máme rozdílné hodnoty    
                suit = _SUITS[index // len(_RANKS)]
                rank = _RANKS[index % len(_RANKS)]
                self.i += 1
                return Card(rank, suit)

In [534]:
deck = reversed(CardDeck())

In [537]:
# for card in deck:
#   print(card)

Nyní iteruje zpětně. V podstatě stačí reversed a poté úprava v next dunder funkci.

### Sekvence

In [547]:
class Squares:
    def __init__(self, length):
        self.squares = [i**2 for i in range(length)]
        
    def __len__(self):
        return len(self.squares)
    
    def __getitem__(self, s):
        return self.squares[s]
    #DELEGUJI GETITEM LISTU - nemusím nic více
    #žádné lt gt > < apod

In [548]:
for num in Squares(5):
    print(num)

0
1
4
9
16


Zajímavé, je že um í i reverzně iterovat. Převezme z listu ale musí mít délku.

In [552]:
class Squares:
    def __init__(self, length):
        self.squares = [i**2 for i in range(length)]
        
    def __len__(self):
        return len(self.squares)
    
    def __getitem__(self, s):
        return self.squares[s]
    
    def __reversed__(self):
        print("Hallo reversed kde jsi?")
        return "Nevím"

In [553]:
for num in reversed(Squares(5)):
    print(num)

Hallo reversed kde jsi?
N
e
v
í
m


Moc pěkné, jelikož neobdrží žádný item, ale obdrží jen string tak jelikož jej lze iterovat tak jej zpětně navrátí :).

## Úskalí při použití funkcí jako argumentu pro Iterátor

In [554]:
import random

In [555]:
class Randoms:
    def __init__(self, n):
        self.n = n
        self.i = 0
            
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            self.i +=1
            return random.randint(0,100)

In [556]:
random.seed(0)
l = list(Randoms(10))
print(l)

[49, 97, 53, 5, 33, 65, 62, 51, 100, 38]


In [557]:
min(l), max(l)

(5, 100)

Použili jsme převod na list, takže je vše ok. Pokud to ale neuděláme:

In [560]:
l = Randoms(10)

In [561]:
min(l)

12

In [562]:
max(l)

ValueError: max() arg is an empty sequence

Proč máme chybu? Protože jsme jej vyčerpali, tzn. jedna funkce jej vyčerpá a v druhé už nic není.